In [1]:
import imdb
import numpy as np
import pandas as pd
import json
from bs4 import BeautifulSoup
import requests
import urllib2
from sklearn.linear_model import LinearRegression as Lin_Reg
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm
import sklearn.preprocessing as Preprocessing
import itertools as it
import math
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import scipy as sp
from itertools import combinations
import unicodedata
import subprocess
import time
from collections import namedtuple
%matplotlib inline

In [33]:
ia = imdb.IMDb()

In [61]:
the_matrix = ia.get_movie('0133093')
the_matrix['plot']

[u"Thomas A. Anderson is a man living two lives. By day he is an average computer programmer and by night a hacker known as Neo. Neo has always questioned his reality, but the truth is far beyond his imagination. Neo finds himself targeted by the police when he is contacted by Morpheus, a legendary computer hacker branded a terrorist by the government. Morpheus awakens Neo to the real world, a ravaged wasteland where most of humanity have been captured by a race of machines that live off of the humans' body heat and electrochemical energy and who imprison their minds within an artificial reality known as the Matrix. As a rebel against the machines, Neo must return to the Matrix and confront the agents: super-powerful computer programs devoted to snuffing out Neo and the entire human rebellion.",
 u'Thomas A. Anderson is a man leading a double life. By day, he is an average computer programmer who pays his taxes and helps his landlady take out the garbage. At night, he is a computer hac

In [51]:
plot_exists = 'the_matrix["plot"]' in locals() or 'the_matrix["plot"]' in globals()

In [62]:
try:
    the_matrix['plot']
except KeyError:
    print "Yay"
else:
    print "Oops"

Oops


In [5]:
movie_df = pd.read_csv('movie_df.csv')

In [7]:
imdb_id_df = pd.read_csv('IMDb IDs.csv')

In [79]:
(str(imdb_id_df.loc[138, 'id']) == '') or (math.isnan(imdb_id_df.loc[138, 'id']))

True

In [69]:
movie_titles = []
directors = []
casts = []
writers = []
plots = []
imdb_ratings = []
imdb_votes = []
runtimes = []

len(movie_titles)

0

In [95]:
for movie_num in range(len(movie_titles), len(imdb_id_df)):
    time.sleep(0.25)
    if ((str(imdb_id_df.loc[movie_num, 'id']) == '') or (math.isnan(imdb_id_df.loc[movie_num, 'id']))):
        movie_titles.append('')
        directors.append('')
        casts.append('')
        writers.append('')
        plots.append('')
        imdb_ratings.append('')
        imdb_votes.append('')
        runtimes.append('')
    else:
        movie = ia.get_movie(str(int(imdb_id_df.loc[movie_num, 'id'])))
        try:
            movie['title']
        except KeyError:
            movie_titles.append('')
        else:
            movie_titles.append(movie['title'])
        try:
            movie['director']
        except KeyError:
            directors.append('')
        else:
            directors.append(movie['director'])
        try:
            movie['cast']
        except KeyError:
            casts.append('')
        else:
            casts.append(movie['cast'])
        try:
            movie['plot']
        except KeyError:
            writers.append('')
        else:
            writers.append(movie['plot'])
        try:
            movie['plot']
        except KeyError:
            plots.append('')
        else:
            plots.append(movie['plot'])
        try:
            movie['rating']
        except KeyError:
            imdb_ratings.append('')
        else:
            imdb_ratings.append(movie['rating'])
        try:
            movie['votes']
        except KeyError:
            imdb_votes.append('')
        else:
            imdb_votes.append(movie['votes'])
        try:
            movie['runtimes']
        except KeyError:
            runtimes.append('')
        else:
            runtimes.append(movie['runtimes'])

In [96]:
len(runtimes)

12000

In [98]:
directors[11234][0]

<Person id:0006969[http] name:_Banks, Elizabeth_>

In [99]:
directors_update = []

for i in range(len(directors)):
    try:
        directors[i][0]
    except IndexError:
        directors_update.append('')
    else:
        directors_update.append(str(directors[i][0]))

In [104]:
casts_update = []

for i in range(len(casts)):
    movie_cast = []
    try:
        casts[i]
    except IndexError:
        casts_update.append('')
    else:
        for j in range(min(len(casts[i]), 5)):
            movie_cast.append(str(casts[i][j]))
        casts_update.append(movie_cast)

In [106]:
writers_update = []

for i in range(len(writers)):
    movie_writer = []
    try:
        writers[i]
    except IndexError:
        writers_update.append('')
    else:
        for j in range(min(len(writers[i]), 5)):
            movie_writer.append(str(writers[i][j]))
        writers_update.append(movie_writer)

UnicodeEncodeError: 'ascii' codec can't encode character u'\xe9' in position 40: ordinal not in range(128)

In [107]:
runtimes_update = []

for i in range(len(directors)):
    try:
        runtimes[i][0]
    except IndexError:
        runtimes_update.append('')
    else:
        runtimes_update.append(str(runtimes[i][0]))

UnicodeEncodeError: 'ascii' codec can't encode character u'\xe9' in position 27: ordinal not in range(128)

In [109]:
final_df = pd.DataFrame(data = {'tmdb_id':movie_df['id'], 
                                  'orig_lang':movie_df['orig_lang'],
                                  'tmdb_overview':movie_df['overview'], 
                                  'tmdb_popularity':movie_df['popularity'],
                                  'release_date':movie_df['release_date'], 
                                  'title':movie_df['title'],
                                  'tmdb_vote_average':movie_df['vote_average'], 
                                  'tmdb_vote_count':movie_df['vote_count'],
                                  'director':directors_update,
                                  'cast':casts_update,
                                  'writer':writers,
                                  'imdb_plot':plots,
                                  'imdb_rating':imdb_ratings,
                                  'imdb_votes':imdb_votes,
                                  'runtime':runtimes,
                         })

In [117]:
final_df.to_csv('Metadata.csv', index = False)

## Features for our Model ##

We have combined both TMDb data and IMDb data for this part of the dataset. From TMDb, we are getting more of the basic info, as well as their website ratings for each movie. From IMDb, we get more of the movie metadata. What we think will be most predictive is looking at directors and the casts of movies. Our thinking is that certain actors and directors usually are associated with a certain type of movie. We also collect the IMDb plot information as hopefully that will also provide us insight into what genre the movie is, through a bag of words analysis. We also collected other information about the movie, like run time and release date.